# Synchronization

## Setup and Helper Functions

In [1]:
from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns
from biopsykit.io.biopac import BiopacDataset
from fau_colors import cmaps
from nilspodlib import Dataset
from packaging.version import Version

from src.empkins_io import EmradDataset
from src.empkins_io.sync import SyncedDataset

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [2]:
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", font="sans-serif", palette=palette)

plt.rcParams["figure.figsize"] = (8, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"

palette

[(0.0, 0.1843137254901961, 0.4235294117647059),
 (0.4666666666666667, 0.6235294117647059, 0.7098039215686275),
 (1.0, 0.7215686274509804, 0.10980392156862745),
 (0.0, 0.6392156862745098, 0.8784313725490196),
 (0.2627450980392157, 0.6901960784313725, 0.16470588235294117),
 (0.7843137254901961, 0.06274509803921569, 0.1803921568627451)]

In [10]:
data_path = Path("data")

radar_path = data_path.joinpath("data_sync-test-01.h5")
biopac_path = data_path.joinpath("data_sync-test-02.acq")
nilspod_path = sorted(data_path.glob("*.bin"))[0]

## Load Data

### Radar

In [7]:
dataset_radar = EmradDataset.from_hd5_file(radar_path)
data_radar = dataset_radar.data_as_df(index="local_datetime", add_sync_in=True, add_sync_out=True)
data_radar = data_radar.xs("rad1", axis=1)

data_radar.head()

channel,I,Q,Sync_In,Sync_Out
date (Europe/Berlin),,,,
2022-11-23 09:25:08+01:00,2130847,-3021788,0,1
2022-11-23 09:25:08.000512+01:00,2130972,-3022018,0,1
2022-11-23 09:25:08.001024+01:00,2130612,-3021858,0,1
2022-11-23 09:25:08.001536+01:00,2130588,-3021996,0,1
2022-11-23 09:25:08.002048+01:00,2130623,-3021736,0,1


### NilsPod

In [8]:
dataset_nilspod = Dataset.from_bin_file(
    nilspod_path, legacy_support="resolve", force_version=Version("0.17.0"), tz="Europe/Berlin"
)
data_nilspod = dataset_nilspod.data_as_df(index="local_datetime")
data_nilspod.head()

,gyr_x,gyr_y,gyr_z,acc_x,acc_y,acc_z,analog_0,analog_1,analog_2
date (Europe/Berlin),,,,,,,,,
2022-11-23 09:28:01.003906+01:00,0.366211,-0.610352,0.061035,0.095801,-0.100591,10.217153,83.0,8.0,25.0
2022-11-23 09:28:01.007812+01:00,0.366211,-0.671387,0.183105,0.114961,-0.100591,10.255474,109.0,9.0,21.0
2022-11-23 09:28:01.011718+01:00,0.427246,-0.671387,0.061035,0.158071,-0.114961,10.245894,128.0,8.0,19.0
2022-11-23 09:28:01.015625+01:00,0.488281,-0.671387,0.061035,0.129331,-0.138911,10.226733,142.0,9.0,17.0
2022-11-23 09:28:01.019531+01:00,0.427246,-0.610352,0.061035,0.086221,-0.148491,10.241104,151.0,7.0,16.0


### Biopac

In [11]:
dataset_biopac = BiopacDataset.from_acq_file(biopac_path)
data_biopac = dataset_biopac.data_as_df(index="local_datetime")

data_biopac.head()

,ecg,rsp,icg_mag,sync,icg_der
date (Europe/Berlin),,,,,
2022-11-23 09:28:06.714000+01:00,-4.780579,-10.000000,49.998474,0.215607,0.834961
2022-11-23 09:28:06.715000037+01:00,-8.030090,-10.000000,49.998474,0.215607,10.377808
2022-11-23 09:28:06.716000074+01:00,-7.139893,-9.887695,49.998474,0.215759,10.037231
2022-11-23 09:28:06.717000111+01:00,-7.467346,-10.000000,49.998474,0.215912,-2.031250
2022-11-23 09:28:06.718000147+01:00,-7.460327,-10.000000,49.998474,0.215912,-19.879150


In [ ]:
synced_dataset = SyncedDataset(sync_type="peak")

synced_dataset.add_dataset(
    "nilspod", data_nilspod, sync_channel_name="analog_1", sampling_rate=dataset_nilspod.info.sampling_rate_hz
)
# synced_dataset.add_dataset(
#    "biopac", data_biopac, sync_channel_name="sync", sampling_rate=dataset_biopac.sampling_rate_hz_sync
# )
synced_dataset.add_dataset(
    "radar", data_radar, sync_channel_name="Sync_In", sampling_rate=dataset_radar.sampling_rate_hz
)

In [ ]:
fig, axs = synced_dataset.plot_sync_channels(figsize=(8, 3))
fig.tight_layout()

In [ ]:
synced_dataset.cut_to_sync_start()

In [ ]:
data_nilspod_cut = synced_dataset.nilspod_cut_
data_radar_cut = synced_dataset.radar_cut_

fig, ax = plt.subplots(figsize=(8, 2))
ax2 = ax.twinx()
data_radar_cut[["Sync_In"]].plot(ax=ax)
# data_biopac[["sync"]].plot(ax=ax)
data_nilspod_cut.filter(like="analog_1").plot(ax=ax2, color="g")
fig.tight_layout()

In [ ]:
synced_dataset.align_datasets("nilspod", cut_to_shortest=True)

In [ ]:
data_nilspod_aligned = synced_dataset.nilspod_aligned_
data_radar_aligned = synced_dataset.radar_aligned_

fig, ax = plt.subplots(figsize=(8, 3))
ax2 = ax.twinx()
data_radar_aligned[["Sync_In"]].plot(ax=ax)
# data_biopac[["sync"]].plot(ax=ax)
data_nilspod_aligned.filter(like="analog_1").plot(ax=ax2, color="g")
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots()
ax2 = ax.twinx()
data_radar[["Sync_In"]].plot(ax=ax)
data_biopac[["sync"]].plot(ax=ax)
data_nilspod.filter(like="analog_1").plot(ax=ax2, color="g")
fig.tight_layout()